# Installing necessary stuff

In [ ]:
!pip install eemont rasterio geopy geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.0 MB/s eta 0:00:00
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=90359400ca0cc933b8d4c23b76c0aed21ea2310fe9ebda4ffaec6135f4ea48ae
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236753 sha256=54757b879196e61af0a0657f481b9725f52cb951afdd7c5bc9cc6995bd852409
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [ ]:
# Import Libraries

import ee  # Google Earth Engine
import eemont  # Extension to Google Earth Engine
import geemap

from PIL import Image
import rasterio
import geopy
import geopy.distance

# Sentinel-2 using Google Earth Engine API

In [ ]:
ee.Authenticate()
ee.Initialize(project='govtechdesign2024')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In some cases, we only know an approximate location of the refugee camp as it can be spread out. Then, it is generally a good idea to define a bounding box from its center and side length

In [ ]:
#@title Define bounding box
import math

def km_to_degrees(km, latitude):
    R = 6371.0
    lat_rad = math.radians(latitude)
    km_per_degree_lon = 111.32 * math.cos(lat_rad)
    km_per_degree_lat = 111.32
    degrees_lat = km / km_per_degree_lat
    degrees_lon = km / km_per_degree_lon

    return degrees_lat, degrees_lon

def calculate_square_corners( center_lon, center_lat, side_length_km):
    half_side_lat, half_side_lon = km_to_degrees(side_length_km/2, center_lat)
    bottom_left = [center_lon - half_side_lon, center_lat - half_side_lat, ]
    top_right = [center_lon + half_side_lon, center_lat + half_side_lat]

    return bottom_left, top_right

# Given parameters
center_lat, center_lon = 11.128772401663994, 42.88745528757827
side_length = 5  # kilometers

bottom_left, top_right = calculate_square_corners(center_lon, center_lat, side_length)

print(f"Bottom-left corner: {bottom_left}")
print(f"Top-right corner: {top_right}")

Bottom-left corner: [42.864567115838476, 11.106314622289219]
Top-right corner: [42.910343459318064, 11.15123018103877]


Sentinel-2 is a great dataset for large-scale applications. However, it has one crucial issue - it has somewhat of a coarse resolution of 10m. Even though it's generally good enough for nature monitoring, it is not the best choice to distinguish individual buildings or tents. In many cases, temporary housing can be way smaller than 10 meters across, meaning it will be sub-pixel size.

As part of its research, Google has developed an Open Buildings dataset. It uses Sentinel-2 data as an input and produces a 4-meter resolution (0.5m raster resolution) building shape data for the Global South regions (South America, Africa, India, etc).

This dataset covers annual data for years 2016 to 2023, which provides an opportunity to study the way refugee camps change over time given conflict intensity changes.


Reference: W. Sirko, E.A. Brempong, J.T.C. Marcos, A. Annkah, A. Korme, M.A. Hassen, K. Sapkota, T. Shekel, A. Diack, S. Nevo, J. Hickey, J.A. Quinn. **High-Resolution Building and Road Detection from Sentinel-2**. arXiv:2310.11622, 2023.

Link: https://sites.research.google/gr/open-buildings/temporal


**Potential list of refugee settlements**:


*   Tongogara Refugee Camp (Kenya) - [32.29002039126818, -20.373715446617975, 32.33792632534928, -20.328799887868424]
*   Tsore Refugee Camp (Ethiopia) - [34.59371583416013, 10.213345954605245, 34.639357795830044, 10.258261513354796]
*   Kiziba Refugee Camp (Rwanda,) - [29.327896708466454, -2.15588440638302, 29.372843422228335, -2.110968847633469]
*   Sahrawi Refugee Camp (Algeria) - [-7.854653286413908, 27.468488528227297, -7.804020460684735, 27.51340408697685]
*   Osire Refugee Camp (Namibia) - [17.342615775067348, -21.095217617298474, 17.3907503538526, -21.050302058548922]
*   Doro Refugee Camp (South Sudan) - [33.72755264683049, 9.959257809490436, 33.773158536550184, 10.004173368239988]
*   Ali Adde Refugee Camp (Djibouti) - [42.864567115838476, 11.106314622289219, 42.910343459318064, 11.15123018103877]

You can also check out overviews on refugee camps in individual countries which are available from UNHCR Database: https://data.unhcr.org/en/search?type%5B0%5D=document

Also, you can check refugee camps locations here: https://data.humdata.org/dataset?q=Refugee+Camps



In [ ]:
geometry = ee.Geometry.Rectangle([42.864567115838476, 11.106314622289219, 42.910343459318064, 11.15123018103877])

In [8]:
#@title Refugee Settlement shape change (2016~2023)
import ee
import geemap
ee.Initialize()

# Load the OpenBuildings temporal dataset
col = ee.ImageCollection('GOOGLE/Research/open-buildings-temporal/v1').filterBounds(geometry)

def add_layers(millis, map_obj):
    mosaic = col.filter(ee.Filter.eq('system:time_start', millis)).mosaic()
    year = ee.Date(millis).get('year').getInfo()

    map_obj.addLayer(
        mosaic.select('building_presence'),
        {'max': 1},
        f'Building Presence Confidence {year}'
    )
timestamps = (
    col.aggregate_array('system:time_start')
    .distinct()
    .sort()
    .getInfo()[:]  # all timestamps, select -N for last N years
)

def mask_s2_clouds(image):
    # Mask clouds using QA60 band for pre-2022, and SCL for 2022 and later
    image_date = ee.Date(image.get('system:time_start'))

    def mask_pre_2021():
        qa = image.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        return qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    def mask_post_2021():
        scl = image.select('SCL')
        cloud_mask = scl.lt(7).Or(scl.gt(10))
        return cloud_mask

    cloud_mask = ee.Algorithms.If(
        image_date.format('YYYY').compareTo('2021').lt(0),
        mask_pre_2021(),
        mask_post_2021()
    )
    return image.updateMask(cloud_mask)

def scale_sentinel_image(image):
    return image.multiply(0.0001)

def process_sentinel(year):
    collection = (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(geometry)
        .filterDate(f'{year}-01-01', f'{year}-12-31')
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
        .map(mask_s2_clouds)
        .map(scale_sentinel_image)
    )
    collection = collection.select(['B4', 'B3', 'B2'])  # Select True Color (RGB) bands
    size = collection.size().getInfo()
    if size == 0:
        return None
    return collection.median()

# Process Sentinel-2 for a specific year
sentinel_year = 2020
sentinel_image = process_sentinel(sentinel_year)

# Create a map object and center on the geometry
map_obj = geemap.Map()
map_obj.centerObject(geometry, 15)
if sentinel_image:
    map_obj.addLayer(
        sentinel_image.clip(geometry),
        {'min': 0, 'max': 0.3, 'gamma': 1.4},
        f'Sentinel-2 True Color {sentinel_year}'
    )
for ts in timestamps:
    add_layers(ts, map_obj)

# Add Bounding Box Layer
map_obj.addLayer(
    ee.Image().byte().paint(geometry, 1, 2),
    {'palette': 'blue'},
    'Bounding Box'
)
map_obj


Map(center=[11.128772690258355, 42.88745528758333], controls=(WidgetControl(options=['position', 'transparent_…

In [9]:
#@title Refugee Camp Size and Area change by yea
def calculate_building_metrics(millis):
    mosaic = col.filter(ee.Filter.eq('system:time_start', millis)).mosaic()
    building_fractional_count = mosaic.select('building_fractional_count')
    binarization_threshold = 0.34
    building_presence = mosaic.select('building_presence').gt(binarization_threshold)
    building_fractional_count = building_fractional_count.multiply(ee.Number(1 * 2).pow(2))

    # Calculate the total building count
    building_count = (building_fractional_count
                      .reduceRegion(
                          reducer=ee.Reducer.sum(),
                          geometry=geometry,
                          scale=1,
                          maxPixels=1e9
                      )
                      .get('building_fractional_count')
                      .getInfo())

    # Calculate the total building area in square meters
    scale_m = 1  # Adjust this for AOI > 50km2
    building_area = (building_presence
                     .reduceRegion(
                         reducer=ee.Reducer.sum(),
                         geometry=geometry,
                         scale=scale_m,
                         maxPixels=1e9
                     )
                     .get('building_presence')
                     .getInfo())

    # Convert building area to square meters (scale ** 2)
    building_area_sqm = building_area * (scale_m ** 2)
    year = ee.Date(millis).get('year').getInfo()
    print(f"Year {year}: Estimated {building_count:.2f} buildings, Total area: {building_area_sqm:.2f} sqm")
    return year, building_count, building_area_sqm

all_timestamps = (
    col.aggregate_array('system:time_start')
    .distinct()
    .sort()
    .getInfo()
)

# Calculate the building count and area for each year
building_metrics = [calculate_building_metrics(ts) for ts in all_timestamps]
print("\nEstimated building metrics by year:")
for year, count, area in building_metrics:
    print(f"{year}: {count:.2f} buildings, {area:.2f} sqm")

Year 2016: Estimated 4187.66 buildings, Total area: 78028.00 sqm
Year 2017: Estimated 4859.05 buildings, Total area: 91014.00 sqm
Year 2018: Estimated 4819.11 buildings, Total area: 81559.00 sqm
Year 2019: Estimated 4835.59 buildings, Total area: 77482.00 sqm
Year 2020: Estimated 4783.42 buildings, Total area: 81090.00 sqm
Year 2021: Estimated 5106.80 buildings, Total area: 83280.00 sqm
Year 2022: Estimated 5058.74 buildings, Total area: 91390.00 sqm
Year 2023: Estimated 5560.65 buildings, Total area: 102330.00 sqm

Estimated building metrics by year:
2016: 4187.66 buildings, 78028.00 sqm
2017: 4859.05 buildings, 91014.00 sqm
2018: 4819.11 buildings, 81559.00 sqm
2019: 4835.59 buildings, 77482.00 sqm
2020: 4783.42 buildings, 81090.00 sqm
2021: 5106.80 buildings, 83280.00 sqm
2022: 5058.74 buildings, 91390.00 sqm
2023: 5560.65 buildings, 102330.00 sqm
